In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
from tensorflow import keras
from keras.callbacks import EarlyStopping

In [33]:
df = pd.read_csv("england-premier-league-matches-2018-to-2019-stats 2.csv")
df = df.sample(frac=1).reset_index(drop=True)

In [34]:
df.drop("home_team_goal_count", axis=1,  inplace=True)
df.drop("away_team_goal_count", axis=1,  inplace=True)


In [230]:
df

,attendance,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,result,Home Team Pre-Match xG,Away Team Pre-Match xG,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win
0,74439,1,0.00,0.00,1.89,1.32,0,0.00,0.00,0.00,0.00,1.37,4.98,9.81
1,51749,1,0.00,0.00,1.32,1.74,1,0.00,0.00,0.00,0.00,4.51,3.77,1.84
2,10353,1,0.00,0.00,1.53,0.74,0,0.00,0.00,0.00,0.00,2.03,3.51,3.96
3,24821,1,0.00,0.00,1.11,1.53,1,0.00,0.00,0.00,0.00,2.31,3.44,3.26
4,24121,1,0.00,0.00,0.47,1.58,1,0.00,0.00,0.00,0.00,7.47,4.27,1.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,53331,38,2.78,1.28,2.79,1.21,0,1.96,1.27,11.89,3.00,1.32,5.80,9.00
376,74457,38,2.00,0.61,1.89,0.74,1,1.67,1.10,9.77,3.50,1.28,6.20,10.00
377,30367,38,1.22,0.33,1.21,0.37,2,1.46,1.11,9.50,4.00,1.41,4.90,7.50
378,60124,38,2.06,1.06,2.00,1.05,2,1.71,1.32,10.39,3.23,2.10,3.50,3.55


In [164]:
normalized_df=(df-df.min())/(df.max()-df.min())

In [35]:
Y = df["result"]
X = df.drop("result", axis=1)

In [ ]:
Y

In [36]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train = np.asarray(X_train).astype('float32')

In [193]:
model = Sequential()
model.add(Dense(8, input_shape=(X.shape[1],), activation='relu'))
model.add(Dense(1, activation='softmax'))
opt = keras.optimizers.Adam()

model.compile(optimizer="Adam", 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
import keras

es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min',
                                   patience=20, 
                                   )

history = model.fit(X_train,
                    Y_train,
                    callbacks=[es],
                    epochs=10,
                    batch_size=100,
                    shuffle=True,
                    validation_split=0.1,
                    verbose=1)

In [51]:
print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [37]:
from xgboost import XGBClassifier
model = XGBClassifier(eval_metric = "mlogloss")
model.fit(X_train, y_train)
y_predict = model.predict_proba(X_test)
print(y_predict)

[[9.8196971e-01 1.7324416e-02 7.0584554e-04]
 [2.0749036e-02 9.6684015e-01 1.2410750e-02]
 [1.7134041e-01 6.0425484e-01 2.2440477e-01]
 [9.9545091e-01 1.2672121e-03 3.2819316e-03]
 [2.7066264e-03 9.9664575e-01 6.4760231e-04]
 [2.9660813e-03 9.7653985e-01 2.0494111e-02]
 [9.9682152e-01 6.4752169e-04 2.5309031e-03]
 [8.9886837e-02 4.3572453e-01 4.7438860e-01]
 [9.3511707e-01 3.5569143e-02 2.9313821e-02]
 [3.6804382e-02 9.4696283e-01 1.6232807e-02]
 [1.2924400e-01 4.1780558e-01 4.5295042e-01]
 [8.4625274e-01 6.8170312e-03 1.4693028e-01]
 [6.6765255e-01 2.5406960e-01 7.8277849e-02]
 [9.9717820e-01 2.0576334e-04 2.6160737e-03]
 [7.5428581e-01 6.7777462e-02 1.7793673e-01]
 [9.3770146e-01 2.1776684e-02 4.0521789e-02]
 [9.9512094e-01 4.3688048e-04 4.4421176e-03]
 [3.1234622e-01 6.8556786e-01 2.0859214e-03]
 [8.8812470e-02 9.0950918e-01 1.6783035e-03]
 [1.7506063e-01 5.6987095e-01 2.5506845e-01]
 [7.7244341e-01 1.9353126e-01 3.4025323e-02]
 [9.7346479e-01 2.0942027e-02 5.5932039e-03]
 [3.470127

In [40]:
y_predict_0 = y_predict[:, 0]
y_predict_1 = y_predict[:, 1]
y_predict_2 = y_predict[:, 2]
#Assigning values to columns and a new file
predicted = pd.DataFrame() #Creates empty dataframe
predicted["Home Win"] = y_predict_0
predicted["Away Win"] = y_predict_1
predicted["Draw"] = y_predict_2
predicted.to_csv("predicted.csv", index = False)
predicted

,Home Win,Away Win,Draw
0,0.981970,0.017324,0.000706
1,0.020749,0.966840,0.012411
2,0.171340,0.604255,0.224405
3,0.995451,0.001267,0.003282
4,0.002707,0.996646,0.000648
...,...,...,...
71,0.818366,0.061125,0.120509
72,0.992116,0.003403,0.004482
73,0.877198,0.021391,0.101410
74,0.024407,0.885910,0.089683


In [42]:
actual = y_test.to_list()
actual

[0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 1]

In [32]:
val = [X_test['odds_ft_home_team_win'].loc[X_test.index[1]], X_test['odds_ft_draw'].loc[X_test.index[1]], X_test['odds_ft_away_team_win'].loc[X_test.index[1]]]
val

[3.15, 3.35, 2.4]

In [49]:
actual = y_test.to_list()
winnings = 0
possibleGuesses = [[0], [1], [2], [0,1], [0,2], [1,2]]
for game in range(0, 76):
    odds = [X_test['odds_ft_home_team_win'].loc[X_test.index[game]], X_test['odds_ft_draw'].loc[X_test.index[game]], X_test['odds_ft_away_team_win'].loc[X_test.index[game]]]
    prediction = y_predict[game].tolist()
    maxExpected = 0
    maxIndex = -1
    for possibleGuess in possibleGuesses:
        expected = 0
        for outcome in possibleGuess:
            #print(outcome, )
            expected += odds[outcome]*prediction[outcome]
        #print(expected, len(possibleGuess))
        expected -= len(possibleGuess)
        if expected > maxExpected:
            maxExpected = expected
            maxIndex = possibleGuesses.index(possibleGuess)
    gainLose = 0
    if possibleGuesses[maxIndex].count(actual[game])>0:
        gainLose = odds[actual[game]]
    gainLose -= len(possibleGuesses[maxIndex])
    print(prediction)
    print("Game " + str(game+1) + ": " + str(gainLose) + ", " + str(possibleGuesses[maxIndex]))
    winnings += gainLose
print(winnings)

[0.9819697141647339, 0.0173244159668684, 0.0007058455375954509]
Game 1: 0.24, [0]
[0.020749036222696304, 0.9668401479721069, 0.012410749681293964]
Game 2: 2.25, [1]
[0.17134040594100952, 0.6042548418045044, 0.22440476715564728]
Game 3: 2.3, [1]
[0.9954509139060974, 0.001267212093807757, 0.0032819316256791353]
Game 4: 0.29000000000000004, [0]
[0.0027066264301538467, 0.9966457486152649, 0.0006476023118011653]
Game 5: -1, [1]
[0.0029660812579095364, 0.9765398502349854, 0.020494110882282257]
Game 6: 3.59, [1]
[0.9968215227127075, 0.0006475216941908002, 0.0025309030897915363]
Game 7: 0.07000000000000006, [0]
[0.08988683670759201, 0.43572452664375305, 0.47438859939575195]
Game 8: -2, [1, 2]
[0.9351170659065247, 0.035569142550230026, 0.029313821345567703]
Game 9: -1, [0]
[0.036804381757974625, 0.946962833404541, 0.01623280718922615]
Game 10: 3.1500000000000004, [1]
[0.12924399971961975, 0.41780558228492737, 0.4529504179954529]
Game 11: 1.5, [1, 2]
[0.8462527394294739, 0.0068170311860740185, 0

#### Earnings simulation

In [295]:
print(round((winnings-76)/76,3)*100)

23.5
